In [8]:
import csv
import scipy
import numpy as np
from util import (
    cognitive_dissonance_score,
    Strength,
    ONEOFFS_TO_TOKEN,
    TOKEN_TO_STRENGTH,
)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
def map_to_strength(strength_str):
    lowercase = strength_str.lower()
    if lowercase in ONEOFFS_TO_TOKEN:
        token = ONEOFFS_TO_TOKEN[lowercase]
    else:
        token = lowercase
    
    return TOKEN_TO_STRENGTH[token]
    
def preprocess(strengths):
    return [map_to_strength(strength_str.strip()) for strength_str in strengths]

with open('survey.csv') as csvfile:
    reader = csv.reader(csvfile)
    user_strengths = {}
    for row in reader:
        if len(row) != 26:
            print("{} has issue".format(row[0]))
        name = row[0]
        evaluator = row[1]
        strengths_in_order = row[2:]
        strengths = preprocess(strengths_in_order)
#         print("{}'s strengths: {}".format(name, strengths))
        
print("Done")

Done


In [81]:
firstrow = None
secondrow = None
thirdrow = None
fourthrow = None
with open('carson.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if firstrow is None:
            firstrow = row
        elif secondrow is None:
            secondrow = row
        elif thirdrow is None:
            thirdrow = row
        elif fourthrow is None:
            fourthrow = row
    

self_perception_scores = np.empty(24)
for i, token in enumerate(firstrow[2:]):
    strength_str = token.strip()
    self_perception_scores[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

external_perception_scores1 = np.empty(24)
for i, token in enumerate(secondrow[2:]):
    strength_str = token.strip()
    external_perception_scores1[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

external_perception_scores2 = np.empty(24)
for i, token in enumerate(thirdrow[2:]):
    strength_str = token.strip()
    external_perception_scores2[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

external_perception_scores3 = np.empty(24)
for i, token in enumerate(fourthrow[2:]):
    strength_str = token.strip()
    external_perception_scores3[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

external_perception_score = (external_perception_scores1 + external_perception_scores2 + external_perception_scores3)/3.0
# print(self_perception_scores)
# print(external_perception_scores1)
# print(external_perception_scores2)
# print(external_perception_scores3)

cog_diss = cognitive_dissonance_score(external_perception_score, self_perception_scores)
s = scipy.spatial.distance.cosine(external_perception_score, self_perception_scores)
print(s)
print(cog_diss)

0.10438157954858163
4.279596331234476


In [82]:
firstrow = None
secondrow = None
thirdrow = None
fourthrow = None
with open('jessica.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if firstrow is None:
            firstrow = row
        elif secondrow is None:
            secondrow = row
        elif thirdrow is None:
            thirdrow = row
        elif fourthrow is None:
            fourthrow = row
    

self_perception_scores = np.empty(24)
for i, token in enumerate(firstrow[2:]):
    strength_str = token.strip()
    self_perception_scores[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

external_perception_scores1 = np.empty(24)
for i, token in enumerate(secondrow[2:]):
    strength_str = token.strip()
    external_perception_scores1[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

s = scipy.spatial.distance.cosine(external_perception_scores1, self_perception_scores)
print(s)
print(cog_diss)

0.08117483811285842
4.279596331234476


In [86]:
firstrow = None
secondrow = None
thirdrow = None
fourthrow = None
with open('anh.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if firstrow is None:
            firstrow = row
        elif secondrow is None:
            secondrow = row
        elif thirdrow is None:
            thirdrow = row
        elif fourthrow is None:
            fourthrow = row
    

self_perception_scores = np.empty(24)
for i, token in enumerate(firstrow[2:]):
    strength_str = token.strip()
    self_perception_scores[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

external_perception_scores1 = np.empty(24)
for i, token in enumerate(secondrow[2:]):
    strength_str = token.strip()
    external_perception_scores1[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

s = scipy.spatial.distance.cosine(self_perception_scores, external_perception_scores1)

s = scipy.spatial.distance.cosine(external_perception_scores1, self_perception_scores)
print(s)
print(cog_diss)

0.1302035152636447
4.279596331234476
